In [ ]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
import pickle
import os

In [ ]:
def scrape_twitter_with_cookies():
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)

    # Load domain first
    driver.get("https://x.com")
    time.sleep(3)

    # Login 
    driver.get("https://x.com/i/flow/login")
    print("Login manually within 30 seconds...")
    time.sleep(30)
    
    # Search by coin name
    search_term = "Bitcoin"
    search_url = f"https://x.com/search?q=%23{search_term}&src=typed_query"
    driver.get(search_url)
    time.sleep(5)

    # Scroll page to load tweets
    for _ in range(2):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

    # Get tweet links
    tweets = driver.find_elements("css selector", "a[href*='/status/']")
    tweet_links = list({tweet.get_attribute("href") for tweet in tweets if '/status' in tweet.get_attribute("href")})
    
    save_path = "C:/Users/Payal Dutta/Documents/twitter_data.txt"

    for link in tweet_links[:5]:
        driver.get(link)
        time.sleep(5)

        try:
            tweet_time = driver.find_element("tag name", "time").get_attribute("datetime")
        except:
            tweet_time = "Time not found"

        texts = driver.find_elements('css selector', "div[data-testid='tweetText']")

        with open(save_path, "a", encoding="utf-8") as f:
            for idx, t in enumerate(texts):
                if idx == 0:
                    f.write(f"\nMain Tweet @ {tweet_time}:\n{t.text}\n")
                else:
                    f.write(f"↳ Comment:\n{t.text}\n")
    
    print(f"Collected {min(len(tweet_links), 5)} tweets and their comments")

    # Logout
    driver.get("https://x.com/logout")
    time.sleep(5)
    try:
        logout_button = driver.find_element("css selector", "div[role='button'][data-testid='confirmationSheetConfirm']")
        logout_button.click()
        time.sleep(3)
        print("Logged out successfully")
    except:
        print("Logout failed or already logged out.")

    driver.quit()


In [ ]:
def scrape_twitter_with_cookies():
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)

    # Load domain first
    driver.get("https://x.com")
    time.sleep(3)

    # Login 
    driver.get("https://x.com/i/flow/login")
    print("login manually")
    time.sleep(30)
    
    driver.get("https://x.com/home")
    time.sleep(5)

    # Search by coin name
    search_term = "Bitcoin"
    search_url = f"https://x.com/search?q=%23{search_term}&src=typed_query"
    driver.get(search_url)
    time.sleep(5)

    # Scroll page
    for _ in range(2):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

    # Scrape tweets
    tweets = driver.find_elements("css selector", "div[data-testid='tweetText']")
    tweet_texts = [tweet.text for tweet in tweets]

    # Save tweets
    save_path_2 = "C:/Users/Payal Dutta/Documents/twitter_data.txt"
    with open(save_path_2, "w", encoding="utf-8") as f:
        for t in tweet_texts:
            f.write(t + "\n")
    print(f"Collected {len(tweet_texts)} tweets")

    # Logout (optional)
    driver.get("https://x.com/logout")
    time.sleep(2)
    try:
        logout_button = driver.find_element("css selector", "div[role='button'][data-testid='confirmationSheetConfirm']")
        logout_button.click()
        print("Logged out successfully")
    except:
        print("Logout failed or already logged out.")

    driver.quit()
